# Previsão de Preços de Carros Usados

Este notebook implementa algoritmos de Machine Learning para prever os preços de carros usados baseado no dataset fornecido pela competição Kaggle.

In [ ]:
import os
import sys

# Adicionar o diretório base do projeto ao sys.path
sys.path.append(os.path.abspath(".."))

# Importar bibliotecas principais
import pandas as pd

from scripts.evaluate import evaluate_model
from scripts.preprocessing import preprocess_data
from scripts.train_model import train_models

# Carregar os dados
train_data = pd.read_csv("../data/train.csv")
test_data = pd.read_csv("../data/test.csv")

# Pré-processamento
X_train, y_train, X_val, y_val, X_test = preprocess_data(
    train_data, test_data, target_column="price", test_size=0.8
)

# Treinamento de modelos
models = train_models(X_train, y_train)

# Encontrar o melhor modelo com base no menor RMSE
best_model_name = min(models, key=lambda x: models[x]["cv_rmse"])
best_model_info = models[best_model_name]
best_model = best_model_info["model"]

print(f"Melhor modelo: {best_model_name}")
print(f"Parâmetros ótimos: {best_model_info.get('best_params', 'N/A')}")
print(f"Cross-Validated RMSE: {best_model_info['cv_rmse']:.4f}")

# Avaliação do melhor modelo
if X_val is not None and y_val is not None:
    print("\nResultados no conjunto de validação:")
    evaluate_model(best_model, X_val, y_val)
else:
    print("\nResultados no conjunto de treino (sem validação):")
    evaluate_model(best_model, X_train, y_train)

# Submissão (usando o melhor modelo)
predictions = best_model.predict(X_test)
submission = pd.DataFrame(
    {
        "id": test_data["id"],
        "price": predictions,
    }
)

submission.to_csv("submission.csv", index=False)
print("Submissão guardada como 'submission.csv'.")